# Modelling a satellite launched towards the sun

### To obtain the different plots with only the Earth, Venus and Earth, etc. exclude the unwanted planets by adding "#" before the line.

In [ ]:
from vpython import *
import numpy as np
import matplotlib. pyplot as plt

### Scene Set-up for modelling

In [ ]:
scene = canvas()

### Defining constants and calculating initial conditions

In [ ]:
G = 6.6704e-11

#Defining the acceleration on one body caused by one other body
def accel(a,b):
    rel_pos = b.pos - a.pos
    return G*b.mass*norm(rel_pos)/rel_pos.mag2 

#Summing the accelerations caused by other bodies on any one body
def totalaccel(a, planets):
    sum_accel = vec(0,0,0)
    for b in planets:
        if (a!=b):
            sum_accel = sum_accel + accel(a,b)
    return sum_accel

#Define constants, position, velocity of the planets and the sun
AU = 149.6e9       
G=6.673e-11

#Mass
sun_mass = 2e30
earth_mass = 6e24
mercury_mass = 3.28e23
venus_mass = 4.87e24
sat_mass = 722

#Radius of orbits
mercury_dist = 0.397*AU
venus_dist = 0.7*AU

#Velocity of planets
earth_vel = 2* np.pi * AU/(365.25 *24. *60.*60.) 
mercury_vel = 2* np.pi * 0.397*AU/(88 *24. *60.*60.) 
venus_vel = 2* np.pi * 0.7*AU/(225 *24. *60.*60.) 


#Mercury's calculation
m_ang = 3.1415 
mpos_x = -np.sin(m_ang)*AU*0.397
mpos_y = np.cos(m_ang)*AU*0.397
mvel_x = np.cos(m_ang)*mercury_vel
mvel_y = np.sin(m_ang)*mercury_vel

#Venus's calculation
v_ang = 2.627
vpos_x = -np.sin(v_ang)*AU*0.7
vpos_y = np.cos(v_ang)*AU*0.7
vvel_x = np.cos(v_ang)*venus_vel
vvel_y = np.sin(v_ang)*venus_vel

In [ ]:
scene.background = color.black
scene.autoscale = 0
scene.range = 2*AU

In [ ]:
#Define the planets and satellite
sun = sphere(pos= vector(0,0,0), velocity = vector(0,0,0),
             mass=sun_mass, radius = 0.005*AU, color =color.yellow)

earth = sphere(pos= vector(-AU, 0, 0), velocity = vector(0,earth_vel,0),
               mass=earth_mass, radius=0.01*AU, color =color.green)
    
sat = sphere(pos=vector(-0.999*AU,0,0), velocity=vector(5000,-5000,0),
            mass=sat_mass, radius=0.01*AU, color=(color.cyan))

mercury = sphere(pos=vector(mpos_x,mpos_y,0),velocity=vector(mvel_x,mvel_y,0),
                 mass=mercury_mass, radius=0.01*AU, color=color.red)

venus = sphere(pos=vector(vpos_x,vpos_y,0),velocity=vector(vvel_x,vvel_y,0),
                 mass=venus_mass, radius=0.01*AU, color=color.yellow)

In [ ]:
#Put planets in the planet list
planets = [sun, earth, mercury, venus, sat]

for b in planets:
    b.accel = vector(0,0,0)
    b.track=curve (color = b.color)

In [ ]:
#Set momentum in the centre of mass to zero, by summing the momenta of all other bodies and calculating velocity of 
#the sun accordingly.

sum=vector(0,0,0)
for b in planets:
    if (b!=sun):
        sum=sum+b.mass*b.velocity

sun.velocity=-sum/sun.mass

#Define the time step dt
dt=60.*60.*3


In [ ]:
#First we calculate the total initial energy in the system by adding kinetic and potential energy
KE_i = 0
PE_i = 0
E_i = 0
for i in planets:
    KE_i = KE_i + (0.5*i.mass*i.velocity.mag2)      #Adding the kinetic energy of each body
    for j in planets:
        if (j != i):
            distance = j.pos - i.pos
            PE_i = PE_i + (6.67e-11*i.mass*j.mass)/(distance).mag  #Adding the gravitational potential energy of each
    E_i = E_i + KE_i + PE_i       #Adding kinetic and potential to get the total initial energy
    
print("Initial Kinetic energy=", KE_i)
print("Initial Potential energy=", PE_i)
print("Total initial energy=", E_i)

### Leapfrog Calculations

In [ ]:
#Initializing leapfrog.
for b in planets:
    b.velocity = b.velocity + totalaccel(b, planets)*dt/2.0


counter = 0

#start leap-frog
while True:
    rate(50)  
    for b in planets:
        b.pos = b.pos + b.velocity*dt
        b.track.append(pos=b.pos)

        b.velocity = b.velocity + totalaccel(b, planets)*dt
    
    counter = counter + 1
    
    scene.center = sun.pos #view centered on the origin


### Checking conservation of energy to estimate error

In [ ]:
#Calculate the total final energy in the system
KE_f = 0
PE_f= 0
E_f = KE_f + PE_f
for i in planets:
    KE_f = KE_f + (0.5*i.mass*i.velocity.mag2)      #Adding the kinetic energy of each
    for j in planets:
        if (j != i):
            distance = j.pos - i.pos
            PE_f = PE_f + (6.67e-11*i.mass*j.mass)/(distance).mag #Adding gravitational potential energy of each body
    E_f = E_f + KE_f + PE_f           #Calculating total final energy as the sum of final kinetic and potential energy
    
print("Final Kinetic energy=", KE_f)
print("Final Potential energy=", PE_f)
print("Total Final energy=", E_f)
print("Difference in total energy is ", ((E_f - E_i)*100)/E_i, "%")